In [1]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from threading import Thread
import pandas as pd
import queue
import datetime

In [4]:
DEFAULT_HISTORIC_DATA_ID=50
DEFAULT_GET_CONTRACT_ID=43

## marker for when queue is finished
FINISHED = object()
STARTED = object()
TIME_OUT = object()

class finishableQueue(object):

    def __init__(self, queue_to_finish):

        self._queue = queue_to_finish
        self.status = STARTED

    def get(self, timeout):
        """
        Returns a list of queue elements once timeout is finished, or a FINISHED flag is received in the queue
        :param timeout: how long to wait before giving up
        :return: list of queue elements
        """
        contents_of_queue=[]
        finished=False

        while not finished:
            try:
                current_element = self._queue.get(timeout=timeout)
                if current_element is FINISHED:
                    finished = True
                    self.status = FINISHED
                else:
                    contents_of_queue.append(current_element)
                    ## keep going and try and get more data

            except queue.Empty:
                ## If we hit a time out it's most probable we're not getting a finished element any time soon
                ## give up and return what we have
                finished = True
                self.status = TIME_OUT


        return contents_of_queue

    def timed_out(self):
        return self.status is TIME_OUT





class TestWrapper(EWrapper):
    """
    The wrapper deals with the action coming back from the IB gateway or TWS instance
    We override methods in EWrapper that will get called when this action happens, like currentTime
    Extra methods are added as we need to store the results in this object
    """

    def __init__(self):
        self._my_contract_details = {}
        self._my_historic_data_dict = {}

    ## error handling code
    def init_error(self):
        error_queue=queue.Queue()
        self._my_errors = error_queue

    def get_error(self, timeout=5):
        if self.is_error():
            try:
                return self._my_errors.get(timeout=timeout)
            except queue.Empty:
                return None

        return None

    def is_error(self):
        an_error_if=not self._my_errors.empty()
        return an_error_if

    def error(self, id, errorCode, errorString):
        ## Overriden method
        errormsg = "IB error id %d errorcode %d string %s" % (id, errorCode, errorString)
        self._my_errors.put(errormsg)


    ## get contract details code
    def init_contractdetails(self, reqId):
        contract_details_queue = self._my_contract_details[reqId] = queue.Queue()

        return contract_details_queue

    def contractDetails(self, reqId, contractDetails):
        ## overridden method

        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(contractDetails)

    def contractDetailsEnd(self, reqId):
        ## overriden method
        if reqId not in self._my_contract_details.keys():
            self.init_contractdetails(reqId)

        self._my_contract_details[reqId].put(FINISHED)

    ## Historic data code
    def init_historicprices(self, tickerid):
        historic_data_queue = self._my_historic_data_dict[tickerid] = queue.Queue()

        return historic_data_queue


    def historicalData(self, tickerid , bar):

        ## Overriden method
        ## Note I'm choosing to ignore barCount, WAP and hasGaps but you could use them if you like
        bardata=(bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume)

        historic_data_dict=self._my_historic_data_dict

        ## Add on to the current data
        if tickerid not in historic_data_dict.keys():
            self.init_historicprices(tickerid)

        historic_data_dict[tickerid].put(bardata)

    def historicalDataEnd(self, tickerid, start:str, end:str):
        ## overriden method

        if tickerid not in self._my_historic_data_dict.keys():
            self.init_historicprices(tickerid)

        self._my_historic_data_dict[tickerid].put(FINISHED)




class TestClient(EClient):
    """
    The client method
    We don't override native methods, but instead call them from our own wrappers
    """
    def __init__(self, wrapper):
        ## Set up with a wrapper inside
        EClient.__init__(self, wrapper)


    def resolve_ib_contract(self, ibcontract, reqId=DEFAULT_GET_CONTRACT_ID):

        """
        From a partially formed contract, returns a fully fledged version
        :returns fully resolved IB contract
        """

        ## Make a place to store the data we're going to return
        contract_details_queue = finishableQueue(self.init_contractdetails(reqId))

        print("Getting full contract details from the server... ")

        self.reqContractDetails(reqId, ibcontract)

        ## Run until we get a valid contract(s) or get bored waiting
        MAX_WAIT_SECONDS = 10
        new_contract_details = contract_details_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if contract_details_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        if len(new_contract_details)==0:
            print("Failed to get additional contract details: returning unresolved contract")
            return ibcontract

        if len(new_contract_details)>1:
            print("got multiple contracts using first one")

        new_contract_details=new_contract_details[0]

        resolved_ibcontract=new_contract_details.contract

        return resolved_ibcontract


    def get_IB_historical_data(self, ibcontract, durationStr="3 Y", barSizeSetting="1 day",whatToShow='TRADES',useRTH=1,
                               tickerid=DEFAULT_HISTORIC_DATA_ID):

        
        # 1 Y => One Year historical
        # 1 day  => Daily
        # TRADES (BID, ASK, MIDPOINT...)
        
        
        """
        Returns historical prices for a contract, up to today
        ibcontract is a Contract
        :returns list of prices in 4 tuples: Open high low close volume
        """


        ## Make a place to store the data we're going to return
        historic_data_queue = finishableQueue(self.init_historicprices(tickerid))

        # Request some historical data. Native method in EClient
        self.reqHistoricalData(
            tickerid,  # tickerId,
            ibcontract,  # contract,
            datetime.datetime.today().strftime("%Y%m%d %H:%M:%S %Z"),  # endDateTime,
            durationStr,  # durationStr,
            barSizeSetting,  # barSizeSetting,
            "TRADES",  # whatToShow,
            1,  # useRTH,
            1,  # formatDate
            False,  # KeepUpToDate <<==== added for api 9.73.2
            [] ## chartoptions not used
        )



        ## Wait until we get a completed data, an error, or get bored waiting
        MAX_WAIT_SECONDS = 10
        print("Getting historical data from the server... could take %d seconds to complete " % MAX_WAIT_SECONDS)

        historic_data = historic_data_queue.get(timeout = MAX_WAIT_SECONDS)

        while self.wrapper.is_error():
            print(self.get_error())

        if historic_data_queue.timed_out():
            print("Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour")

        self.cancelHistoricalData(tickerid)


        return historic_data



class TestApp(TestWrapper, TestClient):
    def __init__(self, ipaddress, portid, clientid):
        TestWrapper.__init__(self)
        TestClient.__init__(self, wrapper=self)

        self.connect(ipaddress, portid, clientid)

        thread = Thread(target = self.run)
        thread.start()

        setattr(self, "_thread", thread)

        self.init_error()


#if __name__ == '__main__':

app = TestApp("127.0.0.1", 4002, 1)

stocks = ['VIX']
historik = pd.DataFrame()
i=49
for stock in stocks:
    i+=1
    ibcontract = IBcontract()
    ibcontract.secType = "IND"       # IND, STK, OPT
#ibcontract.lastTradeDateOrContractMonth="201809"
    ibcontract.symbol=stock
    ibcontract.currency="USD"
    ibcontract.exchange="CBOE"
    resolved_ibcontract=app.resolve_ib_contract(ibcontract)
    historic_data = app.get_IB_historical_data(resolved_ibcontract,tickerid=i)
  
    app.disconnect()
  

Getting full contract details from the server... 
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds.nj
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds
Getting historical data from the server... could take 10 seconds to complete 
IB error id 50 errorcode 162 string Historical Market Data Service error message:No market data permissions for CBOE IND
Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour


In [ ]:
tticker = 'AAPL'

In [21]:
historic_data

[('20180525', 0.16420591, 0.16599973, 0.15688776, 0.15855458, 0),
 ('20180529', 0.15871333, 0.18206473, 0.15871333, 0.17785799, 0),
 ('20180530', 0.17646103, 0.18027091, 0.17433385, 0.17519107, 0),
 ('20180531', 0.17528632, 0.17676265, 0.16958737, 0.17374649, 0),
 ('20180601', 0.1765404, 0.1765404, 0.16504726, 0.16566636, 0),
 ('20180604', 0.16314232, 0.1708097, 0.16312644, 0.16580923, 0),
 ('20180605', 0.16422178, 0.16828566, 0.16247559, 0.16352331, 0),
 ('20180606', 0.16257084, 0.16361855, 0.15853871, 0.16120563, 0),
 ('20180607', 0.16009441, 0.16501551, 0.15922131, 0.16477739, 0),
 ('20180608', 0.16257084, 0.17376236, 0.16257084, 0.16898414, 0),
 ('20180611', 0.17090495, 0.17652453, 0.16893651, 0.17030172, 0),
 ('20180612', 0.17022235, 0.17273052, 0.16517425, 0.16761893, 0),
 ('20180613', 0.16636484, 0.17236541, 0.16347568, 0.16912701, 0),
 ('20180614', 0.16985723, 0.17339725, 0.16472977, 0.16515838, 0),
 ('20180615', 0.16541237, 0.17465134, 0.16390429, 0.16988898, 0),
 ('20180618',

In [12]:
date_list=[]
open_list=[]
high_list=[]
low_list=[]
close_list=[]
volume_list=[]

for i in range (0,len(historic_data)):
    date_list.append(historic_data[i][0])
    open_list.append(historic_data[i][1])
    high_list.append(historic_data[i][2])
    low_list.append(historic_data[i][3])
    close_list.append(historic_data[i][4])
    volume_list.append(historic_data[i][5])

In [13]:
historik['Date']=date_list
historik['Open']=open_list
historik['High']=high_list
historik['Low']=low_list
historik['Close']=close_list
historik['Volume']=volume_list
historik.head()

,Date,Open,High,Low,Close,Volume
0,20140527,11.69,11.84,11.50,11.51,0
1,20140528,11.60,11.86,11.50,11.68,0
2,20140529,11.58,11.82,11.41,11.57,0
3,20140530,11.66,11.70,11.32,11.40,0
4,20140602,11.69,12.17,11.29,11.58,0


In [14]:

historik

,Date,Open,High,Low,Close,Volume
0,20140527,11.69,11.84,11.50,11.51,0
1,20140528,11.60,11.86,11.50,11.68,0
2,20140529,11.58,11.82,11.41,11.57,0
3,20140530,11.66,11.70,11.32,11.40,0
4,20140602,11.69,12.17,11.29,11.58,0
5,20140603,12.03,12.13,11.72,11.87,0
6,20140604,12.15,12.33,11.91,12.08,0
7,20140605,12.09,12.34,11.44,11.68,0
8,20140606,11.32,11.39,10.73,10.73,0
9,20140609,11.23,11.51,10.99,11.15,0


In [15]:
type(historik)

pandas.core.frame.DataFrame

In [16]:
historic_data[87][5]

0

In [17]:
len(historic_data)

1259

In [18]:
historik.to_csv('vix_5Y.csv')
